In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

from sklearn.preprocessing import MinMaxScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/tabular-playground-series-jun-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-jun-2021/train.csv
/kaggle/input/tabular-playground-series-jun-2021/test.csv


In [2]:
train=pd.read_csv("/kaggle/input/tabular-playground-series-jun-2021/train.csv")
train.head()

,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,target
0,0,0,0,6,1,0,0,0,0,7,...,0,0,0,0,0,0,2,0,0,Class_6
1,1,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,1,0,Class_6
2,2,0,0,0,0,0,1,0,3,0,...,0,0,0,0,1,0,0,0,0,Class_2
3,3,0,0,7,0,1,5,2,2,0,...,0,4,0,2,2,0,4,3,0,Class_8
4,4,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Class_2


In [3]:
train.drop(columns=['id'],inplace=True)
train.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,target
0,0,0,6,1,0,0,0,0,7,0,...,0,0,0,0,0,0,2,0,0,Class_6
1,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,1,0,Class_6
2,0,0,0,0,0,1,0,3,0,0,...,0,0,0,0,1,0,0,0,0,Class_2
3,0,0,7,0,1,5,2,2,0,1,...,0,4,0,2,2,0,4,3,0,Class_8
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Class_2


In [4]:
train['target']=train['target'].str.replace('Class_','').astype(int)-1
train.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,target
0,0,0,6,1,0,0,0,0,7,0,...,0,0,0,0,0,0,2,0,0,5
1,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,1,0,5
2,0,0,0,0,0,1,0,3,0,0,...,0,0,0,0,1,0,0,0,0,1
3,0,0,7,0,1,5,2,2,0,1,...,0,4,0,2,2,0,4,3,0,7
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [5]:
X=train.iloc[:,0:-1]
y=train.iloc[:,-1]
X.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74
0,0,0,6,1,0,0,0,0,7,0,...,3,0,0,0,0,0,0,2,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,1,0
2,0,0,0,0,0,1,0,3,0,0,...,8,0,0,0,0,1,0,0,0,0
3,0,0,7,0,1,5,2,2,0,1,...,0,0,4,0,2,2,0,4,3,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=60)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.1, stratify=y_trainval, random_state=20)

In [7]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)
X_train, y_train = np.array(X_train), np.array(y_train)
X_val, y_val = np.array(X_val), np.array(y_val)
X_test, y_test = np.array(X_test), np.array(y_test)

In [8]:
def class_dist(obj):
    count=np.unique(obj,return_counts=True)
    return count
print(class_dist(y_train))

(array([0, 1, 2, 3, 4, 5, 6, 7, 8], dtype=int64), array([ 6565, 17590, 10654,  3387,  2206, 37304, 10633, 37270, 18391]))


In [9]:
class ClassifierDataset(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

train_dataset = ClassifierDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).long())
val_dataset = ClassifierDataset(torch.from_numpy(X_val).float(), torch.from_numpy(y_val).long())
test_dataset = ClassifierDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).long())

In [10]:
target_list = []
for _, t in train_dataset:
    target_list.append(t)
    
target_list = torch.tensor(target_list)
target_list = target_list[torch.randperm(len(target_list))]
print(target_list)

tensor([0, 5, 1,  ..., 2, 2, 7])


In [11]:
class_count = class_dist(y_train)[1:]
class_weights = 1./torch.tensor(class_count, dtype=torch.float)
print(class_weights)

tensor([[1.5232e-04, 5.6850e-05, 9.3861e-05, 2.9525e-04, 4.5331e-04, 2.6807e-05,
         9.4047e-05, 2.6831e-05, 5.4374e-05]])


In [12]:
epochs=300
batch_size=200
hidden=[64,32,16]
learn_rate=0.01
m,n=X_train.shape
k=np.unique(y_train,return_counts=False).shape[0]
print(k)

9


In [13]:
train_loader = DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=1)
test_loader = DataLoader(dataset=test_dataset, batch_size=1)

In [14]:
model=nn.Sequential(nn.Linear(n,hidden[0],bias=True),
                    nn.ReLU(),
                    nn.Linear(hidden[0],hidden[1],bias=True),
                    nn.ReLU(),
                    nn.Linear(hidden[1],hidden[2],bias=True),
                    nn.ReLU(),
                    nn.Linear(hidden[2],k,bias=True))

In [15]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [16]:
model.to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
optimizer = optim.Adam(model.parameters(), lr=learn_rate)
print(model)

Sequential(
  (0): Linear(in_features=75, out_features=64, bias=True)
  (1): ReLU()
  (2): Linear(in_features=64, out_features=32, bias=True)
  (3): ReLU()
  (4): Linear(in_features=32, out_features=16, bias=True)
  (5): ReLU()
  (6): Linear(in_features=16, out_features=9, bias=True)
)


In [17]:
def multi_acc(y_pred, y_test):
    y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
    _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)    
    
    correct_pred = (y_pred_tags == y_test).float()
    acc = correct_pred.sum() / len(correct_pred)
    
    acc = torch.round(acc * 100)
    
    return acc

In [18]:
accuracy_stats = {
    'train': [],
    "val": []
}
loss_stats = {
    'train': [],
    "val": []
}


In [19]:
print("Begin training.")
for e in tqdm(range(1, epochs+1)):
    train_epoch_loss = 0
    train_epoch_acc = 0
    model.train()
    for X_train_batch, y_train_batch in train_loader:
        X_train_batch, y_train_batch = X_train_batch.to(device), y_train_batch.to(device)
        optimizer.zero_grad()
        
        y_train_pred = model(X_train_batch)
        
        train_loss = criterion(y_train_pred, y_train_batch)
        train_acc = multi_acc(y_train_pred, y_train_batch)
        
        train_loss.backward()
        optimizer.step()
        
        train_epoch_loss += train_loss.item()
        train_epoch_acc += train_acc.item()  
    with torch.no_grad():
        
        val_epoch_loss = 0
        val_epoch_acc = 0
        
        model.eval()
        for X_val_batch, y_val_batch in val_loader:
            X_val_batch, y_val_batch = X_val_batch.to(device), y_val_batch.to(device)
            
            y_val_pred = model(X_val_batch)
                        
            val_loss = criterion(y_val_pred, y_val_batch)
            val_acc = multi_acc(y_val_pred, y_val_batch)
            
            val_epoch_loss += val_loss.item()
            val_epoch_acc += val_acc.item()
    loss_stats['train'].append(train_epoch_loss/len(train_loader))
    loss_stats['val'].append(val_epoch_loss/len(val_loader))
    accuracy_stats['train'].append(train_epoch_acc/len(train_loader))
    accuracy_stats['val'].append(val_epoch_acc/len(val_loader))
                              
    
    print(f'Epoch {e+0:03}: | Train Loss: {train_epoch_loss/len(train_loader):.5f} | Val Loss: {val_epoch_loss/len(val_loader):.5f} | Train Acc: {train_epoch_acc/len(train_loader):.3f}| Val Acc: {val_epoch_acc/len(val_loader):.3f}')

Begin training.


  0%|          | 0/300 [00:00<?, ?it/s]

Epoch 001: | Train Loss: 2.12751 | Val Loss: 2.08205 | Train Acc: 27.199| Val Acc: 26.144
Epoch 002: | Train Loss: 2.11209 | Val Loss: 2.05202 | Train Acc: 28.086| Val Acc: 29.856
Epoch 003: | Train Loss: 2.10728 | Val Loss: 2.04318 | Train Acc: 28.201| Val Acc: 26.850
Epoch 004: | Train Loss: 2.10407 | Val Loss: 2.02703 | Train Acc: 27.863| Val Acc: 32.994
Epoch 005: | Train Loss: 2.10168 | Val Loss: 2.08693 | Train Acc: 27.794| Val Acc: 22.400
Epoch 006: | Train Loss: 2.10018 | Val Loss: 2.07293 | Train Acc: 27.374| Val Acc: 27.306
Epoch 007: | Train Loss: 2.09730 | Val Loss: 2.06975 | Train Acc: 26.936| Val Acc: 25.619
Epoch 008: | Train Loss: 2.09534 | Val Loss: 2.06014 | Train Acc: 26.482| Val Acc: 28.444
Epoch 009: | Train Loss: 2.09305 | Val Loss: 2.09470 | Train Acc: 26.729| Val Acc: 21.256
Epoch 010: | Train Loss: 2.09131 | Val Loss: 2.06671 | Train Acc: 26.433| Val Acc: 26.819
Epoch 011: | Train Loss: 2.08751 | Val Loss: 2.03095 | Train Acc: 25.917| Val Acc: 28.575
Epoch 012:

In [20]:
import torch.nn.functional as F
test=pd.read_csv("/kaggle/input/tabular-playground-series-jun-2021/test.csv")
test_id = test['id']
test.drop(columns='id',inplace=True)
with torch.no_grad():
    test = torch.tensor(test.to_numpy()).float()
    if torch.cuda.is_available():
        test = test.cuda()
    test_pred = F.softmax(model(test),dim=1)
    print(test_pred.shape)


test_pred = np.array(test_pred.detach().cpu())
final = pd.DataFrame(test_pred,index=test_id,columns=["Class_"+ str(i+1) for i in range(9)])
final.to_csv("final.csv")

torch.Size([100000, 9])
